# Final Predictions

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() # Creating instance for standard scaler

# Importing Training Dataset

In [5]:
data = pd.read_excel("Inconel_Compiled_Data_New_elems.xlsx")

In [6]:
df = data.copy()

# Separating X and Y variables

In [7]:
x = df.drop(["Type", "Surface Roughness (µm)" , "MRR (mm3/min)"], axis = 1).to_numpy()
y = df[["Surface Roughness (µm)" , "MRR (mm3/min)"]].to_numpy()

# Standardizing X variables

In [8]:
x_scaled = sc.fit_transform(x)

# Training Random Forest (no. of trees = 20) and taking predictions based on providing complete dataset

In [9]:
fr = []

# Model Training and Predictions
lr = RandomForestRegressor(n_estimators=20, random_state=42)
lr.fit(x_scaled,y)

predict = lr.predict(x_scaled)

actual = y

# RMSE Calculation
sr_p_20 = [i[0] for i in predict]
sr_a_20 = [i[0] for i in actual]

mrr_p_20 = [i[1] for i in predict]
mrr_a_20 = [i[1] for i in actual]


summation_sr_20 = 0  #variable to store the summation of differences
summation_mrr_20 = 0

n = len(sr_a_20) #finding total number of items in list

for i in range (0,n):  #looping through each element of the list

    difference_sr_20 = sr_a_20[i] - sr_p_20[i]  #finding the difference between observed and predicted value
    difference_mrr_20 = mrr_a_20[i] - mrr_p_20[i]

    squared_difference_sr_20 = difference_sr_20**2  #taking square of the differences
    squared_difference_mrr_20 = difference_mrr_20**2

    summation_sr_20 = summation_sr_20 + squared_difference_sr_20  #taking a sum of all the differences
    summation_mrr_20 = summation_mrr_20 + squared_difference_mrr_20

MSE_sr_20 = summation_sr_20/n
MSE_mrr_20 = summation_mrr_20/n

RMSE_sr_20 = np.sqrt(MSE_sr_20)
RMSE_mrr_20 = np.sqrt(MSE_mrr_20)


dictionary = {
    "Estimators" : 20,
    "RMSE_SR" : RMSE_sr_20,
    "RMSE_MRR" : RMSE_mrr_20            
    }


fr.append(dictionary)

final_rf_20 = pd.DataFrame(fr)

final_rf_20

,Estimators,RMSE_SR,RMSE_MRR
0,20,0.761094,25.272881


# Creating a dictionary containing grades of Inconel (along with chemical compositions) and their respective combintaions of Current, Time and Voltage

In [10]:
# Reading Chemcical Composition of New Grades File
chem = pd.read_excel("New_Grades_Chem_Comp_Mean.xlsx")
chem.set_index("Type", inplace = True, drop = True)

# Creating a list which contains all the new Inconel grades
lst = list(chem.index)

# Reading all the combinations CSV files
comb = glob.glob(r"Combinations\*.csv")

# Creating a dictionary containing names of Inconel with chemical compositions and their respective combintaions
dict1 = dict(zip(lst, comb))
dict1

{'Inconel 188': 'Combinations\\In188_Combinations.csv',
 'Inconel 230': 'Combinations\\In230_Combinations.csv',
 'Inconel 713 C': 'Combinations\\In713C_Combinations.csv',
 'Inconel 751': 'Combinations\\In751_Combinations.csv',
 'Inconel 792': 'Combinations\\In792_Combinations.csv',
 'Inconel 907': 'Combinations\\In907_Combinations.csv',
 'Inconel 909': 'Combinations\\In909_Combinations.csv',
 'Inconel 925': 'Combinations\\In925_Combinations.csv'}

# Predictions and Summarizing Results of New Grades of Inconel based on Trained RF Model (Trees = 20)

In [11]:
# Iterating over dictionary for each New Inconel grade's chemical compostion and its combinations for Current,
# Time and Voltage

for k,j in dict1.items():
    
    # Saving Inconel Combinations file 
    file = pd.read_csv(r"{}".format(j))
    
    # Locating corresponding name of Inconel 
    composition = chem.loc[k]
    
    # Coverting above variable into DataFrame
    a = pd.DataFrame(composition)
    
    # Transposing and Concatenating Inconel's Chemical Compostion and its combination
    b = a.T
    c= b.reset_index()
    d = pd.concat([c, file], axis = 1)
    
    
    # Filling null values and saving the file
    input_param = d.fillna(method = "ffill")
    input_param.to_csv("{}_input_params.csv".format(k), index = False)
    
    # Standardizing the input parameters
    x1 = input_param.drop("index", axis = 1).to_numpy()
    x = sc.transform(x1)
    
    # Taking Predictions based on input parameters (x), by trained RF model  
    predictions = lr.predict(x)
    
    # Saving the results in a csv file
    df33 = pd.DataFrame(predictions, columns = ["SR", "MRR"])
    df2 = pd.concat([input_param ,df33], axis = 1)
    df3 = df2.iloc[:,-6:]
    df3.to_csv("{}_Results.csv".format(k), index = False )
    
    # Summarizing Unique Values in SR and MRR and ssaving to a csv
    df44 = pd.DataFrame()

    for i in df3["SR"].unique():

        df = df3[df3["SR"] == i]
        df11 = pd.DataFrame()

        for i in df.columns[:4]:

            mn = str(df[i].min())
            mx = str(df[i].max())

            r = "{}-{}".format(mn, mx)

            dict1 = {
                "Range_of_{}".format(i) : r 
            }

            a = pd.DataFrame(dict1, index = [0])
            df11= pd.concat([df11,a],axis =1)


        df22 = pd.DataFrame()

        for i in df.columns[-2:]:

            r = str(df[i].min())

            dict2 = {
            "{}".format(i) : r
            }

            c = pd.DataFrame(dict2, index = [0])
            df22= pd.concat([df22,c],axis =1)

        df33 =pd.concat([df11, df22], axis = 1)

        df44 = pd.concat([df44, df33])
        
    df44.to_excel("{}_Summary.xlsx".format(k), index = False)